# scratchpad for piloting the remove-outliers function

In [63]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.patches as mpatches

from itertools import izip

In [64]:
# this just makes a random dataframe, code stolen from some other notebook
wd = '/home/bay001/projects/tbos_clipseq_20160809/analysis/v3_8-15-2016'
bedhead = ['chrom','start','end','l10pval','l2fc','strand','ann','gene']
regions = ['3utr','5utr','CDS','5utr_and_3utr','intron','noncoding_exon','noncoding_intron','intergenic']
suffix = '.peaks.l2inputnormnew.bed.compressed.bed.annotated'

FOLD_CHANGE_CUTOFF = 3
PVALUE_CUTOFF = 5
import re
def get_counts(wd):
    samples = {}
    for f in glob.glob(os.path.join(wd,'*.annotated')):
        df = pd.read_table(f,names=bedhead)
        df = df[(df['l10pval'] > PVALUE_CUTOFF) & (df['l2fc'] > FOLD_CHANGE_CUTOFF)]
        basename = f.replace(wd+'/','').replace(suffix,'')
        basename = re.sub(r'\.basedon_\w+','',basename)
        samples[basename] = {}
        df['region'] = df.apply(return_region,axis=1)
        for key,value in df['region'].value_counts().iteritems():
            samples[basename][key] = value
        for region in regions:
            if region not in samples[basename]:
                samples[basename][region] = 0
    return samples
def return_region(row):
    region = row['ann'].split('|')[0]
    
    return region
# take out intergenic and 3_and_5prime annotations per discussion 8-25-2016
samples = get_counts(wd)
df = pd.DataFrame(samples)
df.drop('5utr_and_3utr',inplace=True)
df.drop('intergenic',inplace=True)
df = df.T

# this is just messing around with a maps function, but i'm too lazy to make a new notebook :) 
df2 = df.head(4)
df2.columns = range(0,6)
df2

,0,1,2,3,4,5
A11_01_01,1314,156,1562,231,82,62
A11_02_01,1088,263,1247,234,90,72
A1_01_01,2821,42,34,1592,140,62
A1_02_01,3177,45,36,1757,138,65


In [153]:
def remove_outliers(rbpdataframe, conf = 0.95):
    x = 0
    means = list()
    sems = list()
    for key, value in rbpdataframe.iteritems():
        print(key)
        df = rbpdataframe[key].dropna()
        nums = len(df)
        droppercent = (1-conf)/2.0
        dropnum = int(nums*(droppercent))
        df = df.sort_values()
        if(dropnum>0):
            df = df[dropnum:-dropnum]
        
        means.append(df.mean())
        sems.append(df.sem())
    return means, sems

In [154]:
m, s = remove_outliers(df2,0.5)


0
1
2
3
4
5


In [156]:
type(m)

list

In [162]:
np.array(m)[1:]

array([ 116.66666667,  641.5       ,  913.        ,  114.        ,   63.5       ])

In [56]:
pd.concat([df2[df2.columns[:2]],df2[df2.columns[:2]]],axis=1)

,0,1,0,1
A11_01_01,1314,156,1314,156
A11_02_01,1088,263,1088,263
A1_01_01,2821,42,2821,42
A1_02_01,3177,45,3177,45


In [61]:
x = np.array(list(range(0,5)))
y = np.array(list(range(1,6)))

In [62]:
x-y

array([-1, -1, -1, -1, -1])